# Ejercicio 9: Uso de la API de Google Gemini

En este ejercicio vamos a aprender a utilizar la API de OpenAI

## 1. Uso básico

El siguiente código sirve para conectarse con la API de Google Gemini de forma básica

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Gemini_API_KEY")


In [4]:
from google import genai

# 1. Configuramos el cliente usando la variable que ya tienes
client = genai.Client(api_key=secret_value_0)

# 2. Realizamos la consulta al modelo
# Usaremos 'gemini-2.0-flash', que es la versión más reciente y rápida
response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents="¿Cuál es la capital de Ecuador?"
)

# 3. Imprimimos la respuesta
print(response.text)

La capital de Ecuador es **Quito**.


## 2. Retrieval

### 2.1 Cargo el corpus de 20 News Groups

In [5]:
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
newsgroupsdocs = newsgroups.data

In [6]:
import pandas as pd

df = pd.DataFrame(newsgroupsdocs, columns=['text'])
df

,text
0,\n\nI am sure some bashers of Pens fans are pr...
1,My brother is in the market for a high-perform...
2,\n\n\n\n\tFinally you said what you dream abou...
3,\nThink!\n\nIt's the SCSI card doing the DMA t...
4,1) I have an old Jasmine drive which I cann...
...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...
18842,\nNot in isolated ground recepticles (usually ...
18843,I just installed a DX2-66 CPU in a clone mothe...
18844,\nWouldn't this require a hyper-sphere. In 3-...


### 2.2 Transformo a embeddings

In [7]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm

# Eliminar filas sin texto
df = df.dropna(subset=["text"]).reset_index(drop=True)

def normalize_text(s: str) -> str:
    s = str(s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# Aplicamos la normalización
df["text_norm"] = df["text"].map(normalize_text)

In [8]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end].strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

# Generamos la nueva tabla de chunks
records = []
for i, row in tqdm(df.iterrows(), total=len(df), desc="Fragmentando textos"):
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": i,
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
print(f"Total de chunks generados: {len(chunks_df)}")

Fragmentando textos:   0%|          | 0/18846 [00:00<?, ?it/s]

Total de chunks generados: 38871


In [9]:
from sentence_transformers import SentenceTransformer

# Cargamos el modelo (Kaggle lo descargará automáticamente)
MODEL_NAME = "intfloat/e5-base-v2"
model = SentenceTransformer(MODEL_NAME)

# Preparar los textos con el prefijo obligatorio para E5
passages = ["passage: " + t for t in chunks_df["text"].tolist()]

# Generar embeddings
# Si activaste la GPU, esto será muy rápido
embeddings = model.encode(
    passages,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

print("Matriz de embeddings creada con éxito:", embeddings.shape)

2026-01-07 17:34:54.281281: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767807294.507897      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767807294.578857      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767807295.130589      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767807295.130630      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767807295.130633      55 computation_placer.cc:177] computation placer alr

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Batches:   0%|          | 0/608 [00:00<?, ?it/s]

Matriz de embeddings creada con éxito: (38871, 768)


### 2.3 Creo una query y hago la búsqueda

In [15]:
def embed_query(query: str) -> np.ndarray:
    # E5 requiere el prefijo 'query: ' para la búsqueda
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

# Ejemplo de uso:
pregunta_vector = embed_query("Machine learning")


In [17]:
!pip -q install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 84.6 MB/s eta 0:00:00:00:0100:01


In [21]:
# código base para FAISS
import faiss
import numpy as np

# Asumiendo `embeddings` en un array NxD
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

D, I = index.search(pregunta_vector, k=10)

In [26]:
print(I[0][0])

25443


In [39]:
passages[I[0][0]]
#obtener los 10 passages
# se le manda al llm una query donde se muestra el 

"passage: software around. Hopefully I'll be able to provide more information in a future version of this posting. richard welty (welty@balltown.cma.com)"

Obtengo los 5 documentos más similares a mi query

In [ ]:
def get_top_k_indices(query_vec, corpus_embeddings, k=5):
    # Calculamos la similitud (Producto punto es igual a similitud coseno si están normalizados)
    # query_vec tiene forma (1, 768), corpus_embeddings tiene forma (N, 768)
    similarities = np.dot(corpus_embeddings, query_vec.T).flatten()
    
    # Obtenemos los índices de los 'k' valores más altos
    top_k_indices = np.argsort(similarities)[-k:][::-1]
    
    return top_k_indices, similarities[top_k_indices]

In [ ]:
def responder_con_rag(pregunta, k=5):
    # 1. Convertir la pregunta en vector
    query_vec = embed_query(pregunta)
    
    # 2. Buscar los 5 fragmentos más parecidos
    indices, scores = get_top_k_indices(query_vec, embeddings, k=k)
    
    # 3. Recuperar los textos de esos índices
    contextos_recuperados = chunks_df.iloc[indices]["text"].tolist()
    
    # 4. Construir el Prompt para Gemini
    # Unimos los fragmentos en un solo bloque de texto
    contexto_unido = "\n---\n".join(contextos_recuperados)
    
    prompt_final = f"""
    Eres un asistente experto. Utiliza la siguiente información (CONTEXTO) para responder la PREGUNTA del usuario.
    Si la respuesta no está en el contexto, di que no lo sabes, no inventes información.

    CONTEXTO:
    {contexto_unido}

    PREGUNTA:
    {pregunta}
    
    RESPUESTA:
    """

    # 5. Llamada a la API de Gemini
    response = client.models.generate_content(
        model="gemini-3-flash-preview",
        contents=prompt_final
    )
    
    return response.text, contextos_recuperados


mi_pregunta = "¿Qué funciones cumple un indicador de batería?"
respuesta, fuentes = responder_con_rag(mi_pregunta)

print("=== RESPUESTA DE GEMINI ===")
print(respuesta)
print("\n=== FUENTES UTILIZADAS (TOP 1) ===")
print(fuentes[0])